In [1]:
import pandas as pd

In [5]:
import pulp as plp
plp.listSolvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'XPRESS',
 'XPRESS_PY',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD',
 'HiGHS_CMD']

In [225]:
emp = pd.read_csv("employee.csv")
task = pd.read_csv("task_demand.csv")

In [226]:
from pulp import *

In [227]:
emp

,employee,employee_type,task_type
0,r1,normal,task1
1,r2,normal,task2
2,r3,senior,task3
3,r4,senior,task4
4,r5,senior,task5
5,r6,senior,task1
6,r7,normal,task2
7,r8,normal,task3
8,r9,normal,task4
9,r10,normal,task5


In [228]:
task

,task_type,task_demand
0,task1,100
1,task2,40
2,task3,25
3,task4,30
4,task5,20


In [229]:
task_type_list = []
task_type_dict = {}
for i, row in task.iterrows():
    #print(row['task_type'] * int(row['task_demand']))
    task_type_list = task_type_list + [row['task_type']] * int(row['task_demand'])
task_type_dict.update({'task_type':task_type_list})
all_task_df = pd.DataFrame(task_type_dict)
all_task_df['task_id'] = all_task_df.index + 1
all_task_df

,task_type,task_id
0,task1,1
1,task1,2
2,task1,3
3,task1,4
4,task1,5
...,...,...
210,task5,211
211,task5,212
212,task5,213
213,task5,214


In [230]:
all_df = pd.merge(emp,all_task_df,on='task_type')
all_df

,employee,employee_type,task_type,task_id
0,r1,normal,task1,1
1,r1,normal,task1,2
2,r1,normal,task1,3
3,r1,normal,task1,4
4,r1,normal,task1,5
...,...,...,...,...
480,r10,normal,task5,211
481,r10,normal,task5,212
482,r10,normal,task5,213
483,r10,normal,task5,214


In [231]:
all_df.dtypes

employee         object
employee_type    object
task_type        object
task_id           int64
dtype: object

In [232]:
all_df

,employee,employee_type,task_type,task_id
0,r1,normal,task1,1
1,r1,normal,task1,2
2,r1,normal,task1,3
3,r1,normal,task1,4
4,r1,normal,task1,5
...,...,...,...,...
480,r10,normal,task5,211
481,r10,normal,task5,212
482,r10,normal,task5,213
483,r10,normal,task5,214


In [233]:
task

,task_type,task_demand
0,task1,100
1,task2,40
2,task3,25
3,task4,30
4,task5,20


In [234]:
all_df['decision_var'] = [LpVariable("x_"+str(j),cat = "Binary", lowBound = 0) for j in range(all_df.shape[0])]
all_df['deci_str']  = all_df['decision_var'].astype(str)


In [235]:
all_df[(all_df['task_type']=='task1') & (all_df['employee']=='r1')]

,employee,employee_type,task_type,task_id,decision_var,deci_str
0,r1,normal,task1,1,x_0,x_0
1,r1,normal,task1,2,x_1,x_1
2,r1,normal,task1,3,x_2,x_2
3,r1,normal,task1,4,x_3,x_3
4,r1,normal,task1,5,x_4,x_4
...,...,...,...,...,...,...
95,r1,normal,task1,96,x_95,x_95
96,r1,normal,task1,97,x_96,x_96
97,r1,normal,task1,98,x_97,x_97
98,r1,normal,task1,99,x_98,x_98


In [236]:
all_df[(all_df['employee']=='r1')]

,employee,employee_type,task_type,task_id,decision_var,deci_str
0,r1,normal,task1,1,x_0,x_0
1,r1,normal,task1,2,x_1,x_1
2,r1,normal,task1,3,x_2,x_2
3,r1,normal,task1,4,x_3,x_3
4,r1,normal,task1,5,x_4,x_4
...,...,...,...,...,...,...
95,r1,normal,task1,96,x_95,x_95
96,r1,normal,task1,97,x_96,x_96
97,r1,normal,task1,98,x_97,x_97
98,r1,normal,task1,99,x_98,x_98


In [237]:
all_task = task['task_type'].to_list()
all_resource = emp['employee'].to_list()
all_task_id = all_task_df['task_id']


In [238]:
problem = LpProblem('Resource_Allocation', LpMaximize)
problem += lpSum(all_df['decision_var'])
for t in all_task:
    problem += lpSum(all_df[all_df['task_type']==t]['decision_var']) <= int(task[task['task_type']==t]['task_demand'])

#for r in all_resource:
#    for t in all_task_id:
#        problem +=lpSum(all_df[(all_df['task_id']==t) & (all_df['employee']==r)]['decision_var']) == 1


In [239]:
for t in all_task_id:
    problem +=lpSum(all_df[all_df['task_id']==t]['decision_var']) == 1

In [240]:
problem.constraints

OrderedDict([('_C1',
              1*x_0 + 1*x_1 + 1*x_10 + 1*x_100 + 1*x_101 + 1*x_102 + 1*x_103 + 1*x_104 + 1*x_105 + 1*x_106 + 1*x_107 + 1*x_108 + 1*x_109 + 1*x_11 + 1*x_110 + 1*x_111 + 1*x_112 + 1*x_113 + 1*x_114 + 1*x_115 + 1*x_116 + 1*x_117 + 1*x_118 + 1*x_119 + 1*x_12 + 1*x_120 + 1*x_121 + 1*x_122 + 1*x_123 + 1*x_124 + 1*x_125 + 1*x_126 + 1*x_127 + 1*x_128 + 1*x_129 + 1*x_13 + 1*x_130 + 1*x_131 + 1*x_132 + 1*x_133 + 1*x_134 + 1*x_135 + 1*x_136 + 1*x_137 + 1*x_138 + 1*x_139 + 1*x_14 + 1*x_140 + 1*x_141 + 1*x_142 + 1*x_143 + 1*x_144 + 1*x_145 + 1*x_146 + 1*x_147 + 1*x_148 + 1*x_149 + 1*x_15 + 1*x_150 + 1*x_151 + 1*x_152 + 1*x_153 + 1*x_154 + 1*x_155 + 1*x_156 + 1*x_157 + 1*x_158 + 1*x_159 + 1*x_16 + 1*x_160 + 1*x_161 + 1*x_162 + 1*x_163 + 1*x_164 + 1*x_165 + 1*x_166 + 1*x_167 + 1*x_168 + 1*x_169 + 1*x_17 + 1*x_170 + 1*x_171 + 1*x_172 + 1*x_173 + 1*x_174 + 1*x_175 + 1*x_176 + 1*x_177 + 1*x_178 + 1*x_179 + 1*x_18 + 1*x_180 + 1*x_181 + 1*x_182 + 1*x_183 + 1*x_184 + 1*x_185 + 1*x_186 

In [241]:
all_df

,employee,employee_type,task_type,task_id,decision_var,deci_str
0,r1,normal,task1,1,x_0,x_0
1,r1,normal,task1,2,x_1,x_1
2,r1,normal,task1,3,x_2,x_2
3,r1,normal,task1,4,x_3,x_3
4,r1,normal,task1,5,x_4,x_4
...,...,...,...,...,...,...
480,r10,normal,task5,211,x_480,x_480
481,r10,normal,task5,212,x_481,x_481
482,r10,normal,task5,213,x_482,x_482
483,r10,normal,task5,214,x_483,x_483


In [242]:
all_df.sort_values(by='task_id')

,employee,employee_type,task_type,task_id,decision_var,deci_str
0,r1,normal,task1,1,x_0,x_0
100,r6,senior,task1,1,x_100,x_100
101,r6,senior,task1,2,x_101,x_101
1,r1,normal,task1,2,x_1,x_1
2,r1,normal,task1,3,x_2,x_2
...,...,...,...,...,...,...
462,r5,senior,task5,213,x_462,x_462
483,r10,normal,task5,214,x_483,x_483
463,r5,senior,task5,214,x_463,x_463
464,r5,senior,task5,215,x_464,x_464


In [243]:
problem.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tm/0f_tx6g13kl1s2jzyqttkdm40000gn/T/4c2303174f4c417a87e26c76dbb2ca1b-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/tm/0f_tx6g13kl1s2jzyqttkdm40000gn/T/4c2303174f4c417a87e26c76dbb2ca1b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 225 COLUMNS
At line 2651 RHS
At line 2872 BOUNDS
At line 3358 ENDATA
Problem MODEL has 220 rows, 485 columns and 970 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 215 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -215 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were ac

1

In [244]:
status = plp.LpStatus[problem.status]
print(status)

Optimal


In [245]:
print("Optimal overall resouce cost: ",str(plp.value(problem.objective)))


Optimal overall resouce cost:  215.0


In [246]:
decision_vars = {}
for each in problem.variables():
    decision_vars.update({each:str(each.value())})
    #if each.value()==1:
    #    print('XX')
    #
    #    print("Optimal cost of ", each, ": "+str(each.value()))

In [247]:
result = pd.DataFrame(decision_vars.items())
result.columns=['decision_var','allocation']
result['deci_str'] = result['decision_var'].astype(str)
result

,decision_var,allocation,deci_str
0,x_0,1.0,x_0
1,x_1,1.0,x_1
2,x_10,0.0,x_10
3,x_100,0.0,x_100
4,x_101,0.0,x_101
...,...,...,...
480,x_95,1.0,x_95
481,x_96,0.0,x_96
482,x_97,0.0,x_97
483,x_98,1.0,x_98


In [248]:
result[result['deci_str']=='x_0']

,decision_var,allocation,deci_str
0,x_0,1.0,x_0


In [249]:
final = pd.merge(all_df,result,on='deci_str')

In [250]:
final = final[final['allocation']=='1.0']

In [251]:
final = final.drop_duplicates()
final

,employee,employee_type,task_type,task_id,decision_var_x,deci_str,decision_var_y,allocation
0,r1,normal,task1,1,x_0,x_0,x_0,1.0
1,r1,normal,task1,2,x_1,x_1,x_1,1.0
3,r1,normal,task1,4,x_3,x_3,x_3,1.0
4,r1,normal,task1,5,x_4,x_4,x_4,1.0
6,r1,normal,task1,7,x_6,x_6,x_6,1.0
...,...,...,...,...,...,...,...,...
476,r10,normal,task5,207,x_476,x_476,x_476,1.0
477,r10,normal,task5,208,x_477,x_477,x_477,1.0
482,r10,normal,task5,213,x_482,x_482,x_482,1.0
483,r10,normal,task5,214,x_483,x_483,x_483,1.0


In [256]:
final_allocation = final[['task_id','employee','task_type','employee_type']]
final_allocation

,task_id,employee,task_type,employee_type
0,1,r1,task1,normal
1,2,r1,task1,normal
3,4,r1,task1,normal
4,5,r1,task1,normal
6,7,r1,task1,normal
...,...,...,...,...
476,207,r10,task5,normal
477,208,r10,task5,normal
482,213,r10,task5,normal
483,214,r10,task5,normal


In [255]:
final[final['employee']=='r6']

,employee,employee_type,task_type,task_id,decision_var_x,deci_str,decision_var_y,allocation
102,r6,senior,task1,3,x_102,x_102,x_102,1.0
105,r6,senior,task1,6,x_105,x_105,x_105,1.0
109,r6,senior,task1,10,x_109,x_109,x_109,1.0
110,r6,senior,task1,11,x_110,x_110,x_110,1.0
113,r6,senior,task1,14,x_113,x_113,x_113,1.0
114,r6,senior,task1,15,x_114,x_114,x_114,1.0
117,r6,senior,task1,18,x_117,x_117,x_117,1.0
119,r6,senior,task1,20,x_119,x_119,x_119,1.0
123,r6,senior,task1,24,x_123,x_123,x_123,1.0
124,r6,senior,task1,25,x_124,x_124,x_124,1.0


In [254]:
emp

,employee,employee_type,task_type
0,r1,normal,task1
1,r2,normal,task2
2,r3,senior,task3
3,r4,senior,task4
4,r5,senior,task5
5,r6,senior,task1
6,r7,normal,task2
7,r8,normal,task3
8,r9,normal,task4
9,r10,normal,task5
